This code was adapted from the Global Forest Watch Forest Carbon Emissions, Removals, and Net Flux Zonal Statistics GEE code. It outputs annual average forest carbon emissions, removals, and net flux for years 2001-2024. It also outputs annual forest carbon emissions, but users need to exercise caution in comparing annual values, as described in the Global Forest Watch documentation.

We output zonal statistics across Y2Y and thus to calculate statistics for a new region one needs to upload polygons to GEE as a feature collection and run the function on that feature collection.

Original Code: https://code.earthengine.google.com/bc3301dc43c28146d6dc5a316f32ec0f

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Title: Global Forest Watch Forest Emissions, Removals, and Net Flux Zonal Statistics  
Author: Melissa Rose (melissa.rose@wri.org)
Last Modified: 6-05-2025
Currently uses global forest carbon flux model version 1.4.2 (2001-2024)
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
PURPOSE: 
This tool calculates gross carbon emissions, gross carbon removals, and total net flux from forests within the user-provided region(s) of interest (roi). It does this by 
converting carbon emissions, removals, and net flux per hectare rasters (Mg CO2e per ha) to per pixel rasters (Mg CO2e per pixel) and summing per pixel values within the roi. 
Optionally, users may specify if they would also like to calculate annual emissions from forests (Mg CO2e) due to tree cover loss (Hansen et al. 2013) and/ or 
annual averages for emissions, removals, and net flux (Mg CO2e per year) as well as per area rates (Mg CO2e per hectare per year) over the model period of 2001-2024. 

OVERVIEW: 
Before running this tool, please update the information in the 'USER INPUTS' section. Users have the option to modify the following: 
  1. roi: Update with the path to your custom roi ee.FeatureCollection asset. 
  2. filter_roi: Set filter_roi = 'TRUE' if you would like to filter your roi to certain polygons. Otherwise, set filter_roi = 'FALSE'. 
      2a. property_name: If filter_roi = 'TRUE', select which feature property you would like to filter by. To see a list of your features' properties, you can use: 
          print('List of property names:', roi.first().propertyNames())
      2b. property_values: If filter_roi = 'TRUE', select the property values you would like to include: To list all possible values for a property, you can use: 
          print('List of property values:', roi.aggregate_array(property_name).distinct())
  3. annual_emissions: Set annual_emissions = 'TRUE' if you would like to calculate annual emissions due to tree cover loss (Hansen et al. 2013). 
      *Note: If the zonal stats computation is timing out, try setting annual_emissions = 'FALSE' and re-running. 
  6. annual_averages: Set annual_averages = 'TRUE' if you would like to calculate annual averages for emissions, removals, and net flux (Mg CO2 per year) and 
     annual emissions, removals, and net flux per area rates (Mg CO2e per hectare per year).
  7. export_location: Select whether you would like to save the results as an ee.FeatureCollection in GEE ('asset'), a csv in your Google Drive ('drive'), or both ('both'). 
  8. asset_name: Name to give the exported asset.
  9. filename: Name to give the exported csv.
  10. asset_folder: Folder in your GEE asset repo to export the asset to (optional). 
  11. drive_folder: Folder in your Google Drive to export the csv to (optional). 
   
LIMITATIONS: 
Region of Interest Size Limitation: 
  This tool is intended for regions of interest generally smaller than 100 million hectares (Mha). For feature collections with individual features that exceed 100 Mha, 
  it is recommended to split those features into smaller features (less than 100 Mha) before running this script. Actual size limitation will depend on the 
  forest extent within the region(s) of interest. If the computation is timing out, try setting annual_emissions = 'FALSE' and re-running. 

Zonal Stats QAQC with Official GFW Statistics: 
  Zonal statistics calculated on GEE use a different algorithm than zonal statistics calculated on www.globalforestwatch.org so results may differ from GFW's official methods.
  We compared the results from this GEE script with results from GFW's zonal statistics tool using a sample of 1821 features of variable sizes distributed globally. 
  We found that 99% of gross emission, 98% of gross removals, and 96% of net flux results had a percent difference of less than or equal to 5%. Furthermore, we found that
  97% of gross emission, 92% of gross removal, and 90% of net flux results had a difference of less than or equal to 1%. Results with a larger percent difference generally 
  come from regions smaller than 5 hectares. Please treat your results here as an approximation of "correct" results. 

FOR MORE INFORMATION: 
Model described at https://www.nature.com/articles/s41558-020-00976-6
Model updates described at https://essd.copernicus.org/articles/17/1217/2025/
Overview of methods described at https://www.globalforestwatch.org/blog/data-and-research/forest-carbon-flux-data-explained/
Users must familiarize themselves with the data and be aware of its limitations before using this tool. 
Gross emissions data and limitations described at https://data.globalforestwatch.org/datasets/gfw::forest-greenhouse-gas-emissions/about
Gross removals data and limitations described at https://data.globalforestwatch.org/datasets/gfw::forest-carbon-removals/about
Net flux data and limitations described at https://data.globalforestwatch.org/datasets/gfw::forest-greenhouse-gas-net-flux/about

For additional questions about forest carbon flux data, contact David Gibbs (david.gibbs@wri.org).
Please reach out with questions/ feedback regarding this tool to Melissa Rose (melissa.rose@wri.org). 

## Import packages and Initialize GEE

In [2]:
# import packages
import ee
import geemap
import pandas as pd

In [3]:
# authenticate the EE api
ee.Authenticate()

True

In [4]:
# initialize the EE api
ee.Initialize(project='ee-bermane')

In [5]:
# # list all tasks
# tasks = ee.batch.Task.list()
# tasks

# # Cancel all tasks
# for task in tasks:
#     if task.status()['state'] in ['READY', 'RUNNING']:
#         task.cancel()
#         print(f"Cancelled Task ID: {task.id}")

## Sequestration Function Input Examples

In [6]:
# ------------------------------------------------------------------
# USER INPUT EXAMPLES:
# ------------------------------------------------------------------

# Path to user-provided region(s) of interest
roi = ee.FeatureCollection("projects/ee-bermane/assets/y2y_biomes")

# Optional: Clip data to feature collection when working with complex geometries and set roi to rectangle bigger than area ('TRUE' or 'FALSE')
clip_to_fc = False

# Fc to clip to
fc = ee.FeatureCollection("some feature collection")

# Optional: mask data with image when working with raster stats ('TRUE' or 'FALSE')
mask_with_img = False

# img to mask with
img = ee.Image("some image")

# Optional: Filter roi to selected features within the feature collection ('TRUE' or 'FALSE')
filter_roi = False  # Use Python's boolean values instead of strings

# Print list of property names (for debugging)
# print('List of property names:', roi.first().propertyNames().getInfo())

property_name = "ADM0_NAME"

# Print list of property values (for debugging)
# print('List of property values:', roi.aggregate_array(property_name).distinct().getInfo())

property_values = ["Gabon"]

# Optional: Calculate annual emissions due to tree cover loss ('TRUE' or 'FALSE')
annual_emissions = True

# Optional: Calculate average annual emissions, removals, and net flux ('TRUE' or 'FALSE')
annual_averages = True

# Where would you like to export your zonal stats results? ('asset', 'drive', or 'both')
export_location = "drive"
asset_name = ""
filename = "y2y_biomes_carbon_sequestration"
drive_folder = "carbon_sequestration_outputs"

## Define Sequestration Function

In [7]:
# ------------------------------------------------------------------
# DEFINE FUNCTION:
# ------------------------------------------------------------------

def calc_carbon_sequestration(roi,
                              clip_to_fc,
                              mask_with_img,
                              filter_roi,
                              property_name,
                              property_values,
                              annual_emissions,
                              annual_averages,
                              fc='',
                              img=''):
    # export_location,
    # asset_name,
    # filename,
    # drive_folder):
    # ------------------------------------------------------------------
    # PROCESSING
    # ------------------------------------------------------------------

    # STEP 01: Filter ROI using user-provided filter criteria (optional)
    if filter_roi:
        roi = roi.filter(ee.Filter.inList(property_name, property_values))
        print("Region(s) of interest:", roi.getInfo())

    # STEP 02: Import GFW Carbon Layers
    # Mosaic each carbon layer image collection into a single global asset and clip to region(s) of interest
    # revalue to carbon instead of co2e (multiply by 12/44)
    emissions = ee.Image(
        "projects/forma-250/assets/gfw_forest_carbon_gross_emissions/gross_emissions_forest_extent_per_hectare_v1_4_2_2001_2024"
    ).multiply(12/44)

    removals = ee.Image(
        "projects/forma-250/assets/gfw_forest_carbon_gross_removals/gross_removals_forest_extent_per_hectare_v1_4_2_2001_2024"
    ).multiply(12/44)

    netflux = ee.Image(
        "projects/forma-250/assets/gfw_forest_carbon_net_flux/net_flux_forest_extent_per_hectare_v1_4_2_2001_2024"
    ).multiply(12/44)

    # use GEE native pixel area since works same as Hansen but can also be used at other scales
    pixel_area_m2 = ee.Image.pixelArea()

    # still load hansen pixel area for native projection information
    pixel_area_hansen = ee.Image(
    "projects/wri-datalab/hansen_pixel_area"
    )

    # Print per hectare carbon layers to console to view metadata
    # print(
    #     "Emissions per hectare (Mg CO2e/ha between 2001 and 2024):",
    #     emissions.projection().nominalScale().getInfo(),
    # )
    # print(
    #     "Removals per hectare (Mg CO2e/ha between 2001 and 2024):",
    #     removals.projection().nominalScale().getInfo(),
    # )
    # print(
    #     "Net Flux per hectare (Mg CO2e/ha between 2001 and 2024):",
    #     netflux.projection().nominalScale().getInfo(),
    # )
    # print(
    #     "Hansen Pixel Area:",
    #     pixel_area_m2.projection().nominalScale().getInfo(),
    # )

    # STEP 03: Calculate pixel-level emissions, removals, and net flux
    # Convert pixel_area raster from m2 to hectares
    pixel_area_ha = pixel_area_m2.divide(100 * 100).rename(["area__ha"])

    # Create carbon layer mask to filter pixel area raster
    carbon_mask = netflux.mask().neq(0)

    # Mask pixel_area_ha to carbon layers
    pixel_area_flux_model_extent = pixel_area_ha.updateMask(carbon_mask)

    # Save projection information of pixel_area_hansen raster
    proj = pixel_area_hansen.divide(100 * 100).projection().getInfo()
    # proj = pixel_area_ha.projection().getInfo()

    # Calculate per-pixel emissions, removals, and net flux
    pixel_emissions = emissions.multiply(pixel_area_ha)
    pixel_removals = removals.multiply(pixel_area_ha)
    pixel_netflux = netflux.multiply(pixel_area_ha)

    # Stack carbon layers and pixel areas to create 5-band image
    carbon_layers = pixel_emissions.rename(['gfw_forest_carbon_gross_emissions_2001_2024__Mg_C'])\
        .addBands(pixel_removals.rename(['gfw_forest_carbon_gross_removals_2001_2024__Mg_C']))\
        .addBands(pixel_netflux.rename(['gfw_forest_carbon_net_flux_2001_2024__Mg_C']))\
        .addBands(pixel_area_flux_model_extent.rename(['gfw_flux_model_extent__ha']))\
        .addBands(pixel_area_ha)

    # STEP 04: Create emissions raster for each year of tree cover loss and add to carbon_layers (optional)
    if annual_emissions:
        tcl = ee.Image(
            "UMD/hansen/global_forest_change_2024_v1_12").select('lossyear')

        emissions_bands = []
        for year in range(1, 25):
            band_name = f'gfw_forest_carbon_gross_emissions_{2000 + year}__Mg_C'
            emissions_bands.append(pixel_emissions.updateMask(
                tcl.eq(year)).rename(band_name).clip(roi))

        carbon_layers = carbon_layers.addBands(emissions_bands)

    # check scale of carbon layers
    # GEE defines scale and projection based on the output
    # print('carbon_layers_scale:',
    #       carbon_layers.projection().nominalScale().getInfo())

    # STEP 05: Calculate zonal statistics for region(s) of interest
    # clip carbon layers to feature collection (optional)
    if clip_to_fc:
        carbon_layers = carbon_layers.clipToCollection(fc)

    # mask carbon layers with image (optional)
    if mask_with_img:
        carbon_layers = carbon_layers.updateMask(img)

    # Calculate zonal stats for carbon layers by mapping each feature to the reduceRegion function
    zonal_stats = roi.map(lambda feature: feature.set(
        carbon_layers.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=feature.geometry(),
            crs=proj.get('crs'),
            #scale=90,
            crsTransform=proj.get('transform'),
            maxPixels=1e30
        )
    ))

    # STEP 06: Calculate annual averages and per-area rates for emission, removal, and net flux (optional)
    if annual_averages:

        # Number of years within model period: 2001-2024
        model_years = ee.Number(24)

        # Function to calculate emission, removal, and net flux annual averages
        def get_annual_averages(feature):
            gross_emission = ee.Number(feature.get(
                'gfw_forest_carbon_gross_emissions_2001_2024__Mg_C'))
            gross_removals = ee.Number(feature.get(
                'gfw_forest_carbon_gross_removals_2001_2024__Mg_C'))
            total_net_flux = ee.Number(feature.get(
                'gfw_forest_carbon_net_flux_2001_2024__Mg_C'))
            return feature.set({
                'gfw_forest_carbon_average_annual_emissions__Mg(C)_yr-1': gross_emission.divide(model_years),
                'gfw_forest_carbon_average_annual_removals__Mg(C)_yr-1': gross_removals.divide(model_years),
                'gfw_forest_carbon_average_annual_netflux__Mg(C)_yr-1': total_net_flux.divide(model_years)
            })

        # Function to calculate emission, removal, and net flux per area rates
        def get_per_area_rates(feature):
            area = ee.Number(feature.get('gfw_flux_model_extent__ha'))
            return feature.set({
                'gfw_forest_carbon_annual_emission_rate_Mg(C)_ha-1_yr-1': feature.getNumber('gfw_forest_carbon_gross_emissions_2001_2024__Mg_C').divide(area).divide(model_years),
                'gfw_forest_carbon_annual_removal_rate_Mg(C)_ha-1_yr-1': feature.getNumber('gfw_forest_carbon_gross_removals_2001_2024__Mg_C').divide(area).divide(model_years),
                'gfw_forest_carbon_annual_netflux_rate_Mg(C)_ha-1_yr-1': feature.getNumber('gfw_forest_carbon_net_flux_2001_2024__Mg_C').divide(area).divide(model_years)
            })

        zonal_stats = zonal_stats.map(
            get_annual_averages).map(get_per_area_rates)

    # Print final zonal stats
    # print('Zonal Stats:', zonal_stats.getInfo())

    # STEP 08: Export results to drive, asset, or both
    # Cleaning final zonal stats table for export to CSV (removes geometry)
    id_list = ee.List.sequence(0, zonal_stats.size().subtract(1))
    zonal_stats_list = zonal_stats.toList(zonal_stats.size())

    def format_feature(new_sys_index):
        feature = ee.Feature(zonal_stats_list.get(new_sys_index))
        index_string = ee.Number(new_sys_index).format('%01d')
        return feature.set({'system:index': index_string, 'ID': index_string}).setGeometry(None)

    zonal_stats_fc = ee.FeatureCollection(id_list.map(format_feature))

    # return zonal stats and csv
    return zonal_stats, zonal_stats_fc

    # move export tasks outside of function since not working here
    # if export_location == 'drive':
    #     task = ee.batch.Export.table.toDrive(
    #         collection=zonal_stats_csv,
    #         description=filename,
    #         fileNamePrefix=filename,
    #         fileFormat='CSV',
    #         folder=drive_folder
    #     )
    #     task.start()

    # if export_location == 'asset':
    #     task = ee.batch.Export.table.toAsset(
    #         collection=zonal_stats,
    #         description=asset_name,
    #         assetId=asset_name
    #     )
    #     task.start()

    # if export_location == 'both':
    #     drive_task = ee.batch.Export.table.toDrive(
    #         collection=zonal_stats_csv,
    #         description=filename,
    #         fileNamePrefix=filename,
    #         fileFormat='CSV',
    #         folder=drive_folder
    #     )
    #     drive_task.start()

    #     asset_task = ee.batch.Export.table.toAsset(
    #         collection=zonal_stats,
    #         description=asset_name,
    #         assetId=asset_name
    #     )
    #     asset_task.start()

## Export Results to Drive

In [ ]:
# y2y as a whole

# set input parameters
y2y = ee.FeatureCollection("projects/ee-bermane/assets/y2y")
filter_roi = False
clip_to_fc = False
mask_with_img = False
property_name = ""
property_values = [""]
annual_emissions = True
annual_averages = True

# run function
_, zonal_stats_fc = calc_carbon_sequestration(
    roi=y2y,
    clip_to_fc=clip_to_fc,
    mask_with_img = mask_with_img,
    filter_roi=filter_roi,
    property_name=property_name,
    property_values=property_values,
    annual_emissions=annual_emissions,
    annual_averages=annual_averages
)

# export csv to drive
filename = "y2y_carbon_sequestration"
drive_folder = "carbon_sequestration_outputs"

task = ee.batch.Export.table.toDrive(
    collection=zonal_stats_fc,
    description=filename,
    fileNamePrefix=filename,
    fileFormat='CSV',
    folder=drive_folder
)
task.start()

In [ ]:
# biomes

# set input parameters
y2y_biome = ee.FeatureCollection("projects/ee-bermane/assets/y2y_biomes")
filter_roi = False
clip_to_fc = False
mask_with_img = False
property_name = ""
property_values = [""]
annual_emissions = True
annual_averages = True

# run function
_, zonal_stats_fc = calc_carbon_sequestration(
    roi=y2y_biome,
    clip_to_fc=clip_to_fc,
    mask_with_img = mask_with_img,
    filter_roi=filter_roi,
    property_name=property_name,
    property_values=property_values,
    annual_emissions=annual_emissions,
    annual_averages=annual_averages
)

# export csv to drive
filename = "y2y_biomes_carbon_sequestration"
drive_folder = "carbon_sequestration_outputs"

task = ee.batch.Export.table.toDrive(
    collection=zonal_stats_fc,
    description=filename,
    fileNamePrefix=filename,
    fileFormat='CSV',
    folder=drive_folder
)
task.start()

In [ ]:
# ecoregions

# # set input parameters
# y2y_eco = ee.FeatureCollection("projects/ee-bermane/assets/y2y_ecoregions")
# filter_roi = False
# clip_to_fc = False
# mask_with_img = False
# property_name = ""
# property_values = [""]
# annual_emissions = True
# annual_averages = True

# # run function
# _, zonal_stats_csv = calc_carbon_sequestration(
#     roi=y2y_eco,
#     clip_to_fc=clip_to_fc,
#     mask_with_img = mask_with_img,
#     filter_roi=filter_roi,
#     property_name=property_name,
#     property_values=property_values,
#     annual_emissions=annual_emissions,
#     annual_averages=annual_averages
# )

# # export csv to drive
# filename = "y2y_ecoregions_carbon_sequestration"
# drive_folder = "carbon_sequestration_outputs"

# task = ee.batch.Export.table.toDrive(
#     collection=zonal_stats_csv,
#     description=filename,
#     fileNamePrefix=filename,
#     fileFormat='CSV',
#     folder=drive_folder
# )
# task.start()

In [ ]:
# protected areas
y2y_wdpa = ee.FeatureCollection("projects/ee-bermane/assets/PA_y2y_dat_designated_established_area_20250813")
y2y_oecm = ee.FeatureCollection("projects/ee-bermane/assets/WD_OECM_cliptoPA_20250813")

# set input parameters
y2y = ee.FeatureCollection("projects/ee-bermane/assets/y2y")
y2y_protected_areas = y2y_wdpa.merge(y2y_oecm)
filter_roi = False
clip_to_fc = True
fc = y2y_protected_areas
mask_with_img = False
property_name = ""
property_values = [""]
annual_emissions = False
annual_averages = True

# run function
_, zonal_stats_fc = calc_carbon_sequestration(
    roi=y2y,
    clip_to_fc=clip_to_fc,
    fc = fc,
    mask_with_img = mask_with_img,
    filter_roi=filter_roi,
    property_name=property_name,
    property_values=property_values,
    annual_emissions=annual_emissions,
    annual_averages=annual_averages
)

# export csv to drive
filename = "y2y_protected_areas_carbon_sequestration"

task = ee.batch.Export.table.toDrive(
    collection=zonal_stats_fc,
    description=filename,
    fileNamePrefix=filename,
    fileFormat='CSV'
)
task.start()

In [ ]:
# ross river IPCA

# set input parameters
rra = ee.FeatureCollection("projects/ee-bermane/assets/ross_river_ipca")
filter_roi = False
clip_to_fc = False
mask_with_img = False
property_name = ""
property_values = [""]
annual_emissions = True
annual_averages = True

# run function
_, zonal_stats_fc = calc_carbon_sequestration(
    roi=rra,
    filter_roi=filter_roi,
    clip_to_fc=clip_to_fc,
    mask_with_img = mask_with_img,
    property_name=property_name,
    property_values=property_values,
    annual_emissions=annual_emissions,
    annual_averages=annual_averages
)

# export csv to drive
filename = "y2y_rra_carbon_sequestration"
drive_folder = "carbon_sequestration_outputs"

task = ee.batch.Export.table.toDrive(
    collection=zonal_stats_fc,
    description=filename,
    fileNamePrefix=filename,
    fileFormat='CSV',
    folder=drive_folder
)
task.start()

In [8]:
# US/Canada as a whole
# create geometry bigger than us and canada
us_can_geo = ee.Geometry.Rectangle(
    coords=[171, 18, 308, 84],
    proj=ee.Projection('EPSG:4326'),  # Explicitly defining the projection
    geodesic=False)

# create a Feature with this geometry
us_can_feature = ee.Feature(us_can_geo, {"name": "Bigger than US/Canada"})

# set input parameters
us_can_fc = ee.FeatureCollection(us_can_feature)
filter_roi = False
clip_to_fc = True
fc = ee.FeatureCollection("projects/ee-bermane/assets/us_can_simple")
mask_with_img = False
property_name = ""
property_values = [""]
annual_emissions = True
annual_averages = True

# run function
_, zonal_stats_fc = calc_carbon_sequestration(
    roi=us_can_fc,
    clip_to_fc=clip_to_fc,
    fc = fc,
    mask_with_img = mask_with_img,
    filter_roi=filter_roi,
    property_name=property_name,
    property_values=property_values,
    annual_emissions=annual_emissions,
    annual_averages=annual_averages
)

# export csv to drive
filename = "us_can_carbon_sequestration"

task = ee.batch.Export.table.toDrive(
    collection=zonal_stats_fc,
    description=filename,
    fileNamePrefix=filename,
    fileFormat='CSV'
)
task.start()

In [25]:
# US/Canada in K1
# create geometry bigger than us and canada
us_can_geo = ee.Geometry.Rectangle(
    coords=[171, 18, 308, 84],
    proj=ee.Projection('EPSG:4326'),  # Explicitly defining the projection
    geodesic=False)

# create a Feature with this geometry
us_can_feature = ee.Feature(us_can_geo, {"name": "Bigger than US/Canada"})

# set input parameters
us_can_fc = ee.FeatureCollection(us_can_feature)
filter_roi = False
clip_to_fc = True
fc = ee.FeatureCollection("projects/ee-bermane/assets/us_can_simple")
mask_with_img = True
img = ee.Image("projects/ee-bermane/assets/k1_global")
property_name = ""
property_values = [""]
annual_emissions = True
annual_averages = True

# run function
_, zonal_stats_fc = calc_carbon_sequestration(
    roi=us_can_fc,
    clip_to_fc=clip_to_fc,
    fc = fc,
    mask_with_img = mask_with_img,
    img=img,
    filter_roi=filter_roi,
    property_name=property_name,
    property_values=property_values,
    annual_emissions=annual_emissions,
    annual_averages=annual_averages
)

# export csv to drive
filename = "us_can_k1_carbon_sequestration"

task = ee.batch.Export.table.toDrive(
    collection=zonal_stats_fc,
    description=filename,
    fileNamePrefix=filename,
    fileFormat='CSV'
)
task.start()

In [26]:
# US/Canada in Not K1 (good for sanity check)
# create geometry bigger than us and canada
us_can_geo = ee.Geometry.Rectangle(
    coords=[171, 18, 308, 84],
    proj=ee.Projection('EPSG:4326'),  # Explicitly defining the projection
    geodesic=False)

# create a Feature with this geometry
us_can_feature = ee.Feature(us_can_geo, {"name": "Bigger than US/Canada"})

# set input parameters
us_can_fc = ee.FeatureCollection(us_can_feature)
filter_roi = False
clip_to_fc = True
fc = ee.FeatureCollection("projects/ee-bermane/assets/us_can_simple")
mask_with_img = True
img = ee.Image("projects/ee-bermane/assets/k1_global").Not()
property_name = ""
property_values = [""]
annual_emissions = True
annual_averages = True

# run function
_, zonal_stats_fc = calc_carbon_sequestration(
    roi=us_can_fc,
    clip_to_fc=clip_to_fc,
    fc = fc,
    mask_with_img = mask_with_img,
    img=img,
    filter_roi=filter_roi,
    property_name=property_name,
    property_values=property_values,
    annual_emissions=annual_emissions,
    annual_averages=annual_averages
)

# export csv to drive
filename = "us_can_not_k1_carbon_sequestration"

task = ee.batch.Export.table.toDrive(
    collection=zonal_stats_fc,
    description=filename,
    fileNamePrefix=filename,
    fileFormat='CSV'
)
task.start()

In [7]:
# US/Canada in K3
# create geometry bigger than us and canada
us_can_geo = ee.Geometry.Rectangle(
    coords=[171, 18, 308, 84],
    proj=ee.Projection('EPSG:4326'),  # Explicitly defining the projection
    geodesic=False)

# create a Feature with this geometry
us_can_feature = ee.Feature(us_can_geo, {"name": "Bigger than US/Canada"})

# set input parameters
us_can_fc = ee.FeatureCollection(us_can_feature)
filter_roi = False
clip_to_fc = True
fc = ee.FeatureCollection("projects/ee-bermane/assets/us_can_simple")
mask_with_img = True
img = ee.Image("projects/ee-bermane/assets/k3_binary").unmask()
property_name = ""
property_values = [""]
annual_emissions = True
annual_averages = True

# run function
_, zonal_stats_fc = calc_carbon_sequestration(
    roi=us_can_fc,
    clip_to_fc=clip_to_fc,
    fc = fc,
    mask_with_img = mask_with_img,
    img=img,
    filter_roi=filter_roi,
    property_name=property_name,
    property_values=property_values,
    annual_emissions=annual_emissions,
    annual_averages=annual_averages
)

# export csv to drive
filename = "us_can_k3_carbon_sequestration"

task = ee.batch.Export.table.toDrive(
    collection=zonal_stats_fc,
    description=filename,
    fileNamePrefix=filename,
    fileFormat='CSV'
)
task.start()

In [8]:
# US/Canada in Not K3 (good for sanity check)
# create geometry bigger than us and canada
us_can_geo = ee.Geometry.Rectangle(
    coords=[171, 18, 308, 84],
    proj=ee.Projection('EPSG:4326'),  # Explicitly defining the projection
    geodesic=False)

# create a Feature with this geometry
us_can_feature = ee.Feature(us_can_geo, {"name": "Bigger than US/Canada"})

# set input parameters
us_can_fc = ee.FeatureCollection(us_can_feature)
filter_roi = False
clip_to_fc = True
fc = ee.FeatureCollection("projects/ee-bermane/assets/us_can_simple")
mask_with_img = True
img = ee.Image("projects/ee-bermane/assets/k3_binary").unmask().Not()
property_name = ""
property_values = [""]
annual_emissions = True
annual_averages = True

# run function
_, zonal_stats_fc = calc_carbon_sequestration(
    roi=us_can_fc,
    clip_to_fc=clip_to_fc,
    fc = fc,
    mask_with_img = mask_with_img,
    img=img,
    filter_roi=filter_roi,
    property_name=property_name,
    property_values=property_values,
    annual_emissions=annual_emissions,
    annual_averages=annual_averages
)

# export csv to drive
filename = "us_can_not_k3_carbon_sequestration"

task = ee.batch.Export.table.toDrive(
    collection=zonal_stats_fc,
    description=filename,
    fileNamePrefix=filename,
    fileFormat='CSV'
)
task.start()

In [11]:
# Canada as a whole
# create geometry bigger than us and canada
us_can_geo = ee.Geometry.Rectangle(
    coords=[171, 18, 308, 84],
    proj=ee.Projection('EPSG:4326'),  # Explicitly defining the projection
    geodesic=False)

# create a Feature with this geometry
us_can_feature = ee.Feature(us_can_geo, {"name": "Bigger than US/Canada"})

# load canada as a fc
countries = ee.FeatureCollection("USDOS/LSIB/2017")
canada = countries.filter(ee.Filter.eq('COUNTRY_NA', 'Canada'))

# set input parameters
us_can_fc = ee.FeatureCollection(us_can_feature)
filter_roi = False
clip_to_fc = True
fc = canada
mask_with_img = False
property_name = ""
property_values = [""]
annual_emissions = True
annual_averages = True

# run function
_, zonal_stats_fc = calc_carbon_sequestration(
    roi=us_can_fc,
    clip_to_fc=clip_to_fc,
    fc = fc,
    mask_with_img = mask_with_img,
    filter_roi=filter_roi,
    property_name=property_name,
    property_values=property_values,
    annual_emissions=annual_emissions,
    annual_averages=annual_averages
)

# export csv to drive
filename = "can_carbon_sequestration"

task = ee.batch.Export.table.toDrive(
    collection=zonal_stats_fc,
    description=filename,
    fileNamePrefix=filename,
    fileFormat='CSV'
)
task.start()

In [8]:
# Global
# create a global geometry to reduce over
globe_geo = ee.Geometry.Rectangle(
    coords=[-180, -90, 180, 90],
    proj=ee.Projection('EPSG:4326'),  # Explicitly defining the projection,
    geodesic=False)

# Create a Feature with this geometry
globe_f = ee.Feature(globe_geo, {"name": "Global"})

# set input parameters
globe_fc = ee.FeatureCollection(globe_f)
filter_roi = False
clip_to_fc = False
mask_with_img = False
property_name = ""
property_values = [""]
annual_emissions = True
annual_averages = True

# run function
_, zonal_stats_fc = calc_carbon_sequestration(
    roi=globe_fc,
    clip_to_fc=clip_to_fc,
    mask_with_img = mask_with_img,
    filter_roi=filter_roi,
    property_name=property_name,
    property_values=property_values,
    annual_emissions=annual_emissions,
    annual_averages=annual_averages
)

# export csv to drive
filename = "global_carbon_sequestration"

task = ee.batch.Export.table.toDrive(
    collection=zonal_stats_fc,
    description=filename,
    fileNamePrefix=filename,
    fileFormat='CSV'
)
task.start()

In [23]:
# Global K1
# create a global geometry to reduce over
globe_geo = ee.Geometry.Rectangle(
    coords=[-180, -90, 180, 90],
    proj=ee.Projection('EPSG:4326'),  # Explicitly defining the projection,
    geodesic=False)

# Create a Feature with this geometry
globe_f = ee.Feature(globe_geo, {"name": "Global"})

# set input parameters
globe_fc = ee.FeatureCollection(globe_f)
filter_roi = False
clip_to_fc = False
mask_with_img = True
img = ee.Image("projects/ee-bermane/assets/k1_global")
property_name = ""
property_values = [""]
annual_emissions = True
annual_averages = True

# run function
_, zonal_stats_fc = calc_carbon_sequestration(
    roi=globe_fc,
    clip_to_fc=clip_to_fc,
    mask_with_img = mask_with_img,
    img=img,
    filter_roi=filter_roi,
    property_name=property_name,
    property_values=property_values,
    annual_emissions=annual_emissions,
    annual_averages=annual_averages
)

# export csv to drive
filename = "global_k1_carbon_sequestration"

task = ee.batch.Export.table.toDrive(
    collection=zonal_stats_fc,
    description=filename,
    fileNamePrefix=filename,
    fileFormat='CSV'
)
task.start()

In [24]:
# Global Not K1 (good for sanity check)
# create a global geometry to reduce over
globe_geo = ee.Geometry.Rectangle(
    coords=[-180, -90, 180, 90],
    proj=ee.Projection('EPSG:4326'),  # Explicitly defining the projection,
    geodesic=False)

# Create a Feature with this geometry
globe_f = ee.Feature(globe_geo, {"name": "Global"})

# set input parameters
globe_fc = ee.FeatureCollection(globe_f)
filter_roi = False
clip_to_fc = False
mask_with_img = True
img = ee.Image("projects/ee-bermane/assets/k1_global").Not()
property_name = ""
property_values = [""]
annual_emissions = True
annual_averages = True

# run function
_, zonal_stats_fc = calc_carbon_sequestration(
    roi=globe_fc,
    clip_to_fc=clip_to_fc,
    mask_with_img = mask_with_img,
    img=img,
    filter_roi=filter_roi,
    property_name=property_name,
    property_values=property_values,
    annual_emissions=annual_emissions,
    annual_averages=annual_averages
)

# export csv to drive
filename = "global_not_k1_carbon_sequestration"

task = ee.batch.Export.table.toDrive(
    collection=zonal_stats_fc,
    description=filename,
    fileNamePrefix=filename,
    fileFormat='CSV'
)
task.start()

In [9]:
# Global K3
# create a global geometry to reduce over
globe_geo = ee.Geometry.Rectangle(
    coords=[-180, -90, 180, 90],
    proj=ee.Projection('EPSG:4326'),  # Explicitly defining the projection,
    geodesic=False)

# Create a Feature with this geometry
globe_f = ee.Feature(globe_geo, {"name": "Global"})

# set input parameters
globe_fc = ee.FeatureCollection(globe_f)
filter_roi = False
clip_to_fc = False
mask_with_img = True
img = ee.Image("projects/ee-bermane/assets/k3_binary").unmask()
property_name = ""
property_values = [""]
annual_emissions = True
annual_averages = True

# run function
_, zonal_stats_fc = calc_carbon_sequestration(
    roi=globe_fc,
    clip_to_fc=clip_to_fc,
    mask_with_img = mask_with_img,
    img=img,
    filter_roi=filter_roi,
    property_name=property_name,
    property_values=property_values,
    annual_emissions=annual_emissions,
    annual_averages=annual_averages
)

# export csv to drive
filename = "global_k3_carbon_sequestration"

task = ee.batch.Export.table.toDrive(
    collection=zonal_stats_fc,
    description=filename,
    fileNamePrefix=filename,
    fileFormat='CSV'
)
task.start()

In [10]:
# Global Not K3 (good for sanity check)
# create a global geometry to reduce over
globe_geo = ee.Geometry.Rectangle(
    coords=[-180, -90, 180, 90],
    proj=ee.Projection('EPSG:4326'),  # Explicitly defining the projection,
    geodesic=False)

# Create a Feature with this geometry
globe_f = ee.Feature(globe_geo, {"name": "Global"})

# set input parameters
globe_fc = ee.FeatureCollection(globe_f)
filter_roi = False
clip_to_fc = False
mask_with_img = True
img = ee.Image("projects/ee-bermane/assets/k3_binary").unmask().Not()
property_name = ""
property_values = [""]
annual_emissions = True
annual_averages = True

# run function
_, zonal_stats_fc = calc_carbon_sequestration(
    roi=globe_fc,
    clip_to_fc=clip_to_fc,
    mask_with_img = mask_with_img,
    img=img,
    filter_roi=filter_roi,
    property_name=property_name,
    property_values=property_values,
    annual_emissions=annual_emissions,
    annual_averages=annual_averages
)

# export csv to drive
filename = "global_not_k3_carbon_sequestration"

task = ee.batch.Export.table.toDrive(
    collection=zonal_stats_fc,
    description=filename,
    fileNamePrefix=filename,
    fileFormat='CSV'
)
task.start()

In [9]:
# compute zonal statistics across ecosystems
# first define ecosystem masks

# load layers
y2y_biomes = ee.FeatureCollection("projects/ee-bermane/assets/y2y_biomes")
lc = ee.Image("USGS/NLCD_RELEASES/2020_REL/NALCMS")
peat = ee.Image("projects/sat-io/open-datasets/GLOBAL-PEATLAND-DATABASE")

# Rasterize y2y_biomes to use for ecosystem classification based on BIOME_ID:
biomes_raster = y2y_biomes.reduceToImage(
    properties=['BIOME_ID'], reducer=ee.Reducer.first())

# TEMPERATE FORESTS --> FORESTRY
# create a mask based on forest lc types
# and not peatlands (1, 2)
# and biomes mask (5, 8, 13)
temperate_forest_mask = (
    lc.gte(1)
    .And(lc.lte(6))
    .And(peat.mask().Not())
    .And(biomes_raster.eq(5)
         .Or(biomes_raster.eq(8))
         .Or(biomes_raster.eq(13)))
)

# NORTHERN FORESTS --> FORESTRY
# create a mask based on forest lc types
# and not peatlands (1, 2)
# and biomes mask (6, 11)
northern_forest_mask = (
    lc.gte(1)
    .And(lc.lte(6))
    .And(peat.mask().Not())
    .And(biomes_raster.eq(6)
         .Or(biomes_raster.eq(11)))
)

# TEMPERATE GRASSLANDS AND SHRUBLANDS --> AGRICULTURE
# create a mask based on grasslands and shrublands LC type (8, 10)
# and not peatlands (1, 2)
# and biomes mask (8)
temperate_grass_shrub_mask = (
    (lc.eq(8)
     .Or(lc.eq(10)))
    .And(peat.mask().Not())
    .And(biomes_raster.eq(8))
)

# MONTANE GRASSLANDS AND SHRUBLANDS --> AGRICULTURE
# create a mask based on grasslands and shrublands LC type (8, 10)
# and not peatlands (1, 2)
# and biomes mask (5)
montane_grass_shrub_mask = (
    (lc.eq(8)
     .Or(lc.eq(10)))
    .And(peat.mask().Not())
    .And(biomes_raster.eq(5))
)

# NORTHERN GRASSLANDS AND SHRUBLANDS --> MINING/OIL/GAS/DEV
# create a mask based on grasslands, shrublands, lichen-moss LC type (8, 10, 11, 12, 13)
# and not peatlands (1, 2)
# and biomes mask (6, 11)
northern_grass_shrub_mask = (
    (lc.eq(8)
     .Or(lc.eq(10))
     .Or(lc.eq(11))
     .Or(lc.eq(12))
     .Or(lc.eq(13)))
    .And(peat.mask().Not())
    .And(biomes_raster.eq(6)
         .Or(biomes_raster.eq(11)))
)

# TEMPERATE WETLANDS --> AGRICULTURE
# create a mask based on wetlands LC type (14)
# and not peatlands (1, 2)
# and biomes mask (5, 8)
temperate_wetlands_mask = (
    lc.eq(14)
    .And(peat.mask().Not())
    .And(biomes_raster.eq(5)
         .Or(biomes_raster.eq(8)))
)

# NORTHERN WETLANDS --> MINING/OIL/GAS/DEV
# create a mask based on wetlands LC type (14)
# and not peatlands (1, 2)
# and biomes mask (6, 11)
northern_wetlands_mask = (
    lc.eq(14)
    .And(peat.mask().Not())
    .And(biomes_raster.eq(6)
         .Or(biomes_raster.eq(11)))
)

# TEMPERATE PEATLANDS --> AGRICULTURE
# create a mask based on peatlands LC type (1, 2)
# and biomes mask (5, 8)
temperate_peatlands_mask = (
    peat.mask()
    .And(biomes_raster.eq(5)
         .Or(biomes_raster.eq(8)))
)

# NORTHERN PEATLANDS --> MINING/OIL/GAS/DEV
# create a mask based on peatlands LC type (1, 2)
# and biomes mask (6, 11)
northern_peatlands_mask = (
    peat.mask()
    .And(biomes_raster.eq(6)
         .Or(biomes_raster.eq(11)))
)

# TEMPERATE BARREN LANDS --> MINING/OIL/GAS/DEV
# LC type (16)
# and not peatlands (1, 2)
# and biomes mask (5, 8)
temperate_barren_mask = (
    lc.eq(16)
    .And(peat.mask().Not())
    .And(biomes_raster.eq(5)
         .Or(biomes_raster.eq(8)))
)

# NORTHERN BARREN LANDS --> MINING/OIL/GAS/DEV
# create a mask based on barren lands LC type (16)
# and not peatlands (1, 2)
# and biomes mask (6, 11)
northern_barren_mask = (
    lc.eq(16)
    .And(peat.mask().Not())
    .And(biomes_raster.eq(6)
         .Or(biomes_raster.eq(11)))
)

# check masks to make sure they encompass full extent
mask_check = (
    temperate_forest_mask
    .Or(northern_forest_mask)
    .Or(temperate_grass_shrub_mask)
    .Or(montane_grass_shrub_mask)
    .Or(northern_grass_shrub_mask)
    .Or(temperate_wetlands_mask)
    .Or(northern_wetlands_mask)
    .Or(temperate_peatlands_mask)
    .Or(northern_peatlands_mask)
    .Or(temperate_barren_mask)
    .Or(northern_barren_mask)
)

# Non-manageable systems
# DESERT --> NOT MANAGEABLE
# biomes mask (13)
# and not peatlands (1, 2)
# and not forest (1-6)
desert_mask = (
    lc.gte(7)
    .And(peat.mask().Not())
    .And(biomes_raster.eq(13))
)

# Vegetation cover for biomass replaceability in peatlands
# TEMPERATE FORESTS --> FORESTRY
# create a mask based on forest lc types
# and biomes mask (5, 8, 13)
temperate_forest_bio_mask = (
    lc.gte(1)
    .And(lc.lte(6))
    .And(biomes_raster.eq(5)
         .Or(biomes_raster.eq(8))
         .Or(biomes_raster.eq(13)))
)

# NORTHERN FORESTS --> FORESTRY
# create a mask based on forest lc types
# and biomes mask (6, 11)
northern_forest_bio_mask = (
    lc.gte(1)
    .And(lc.lte(6))
    .And(biomes_raster.eq(6)
         .Or(biomes_raster.eq(11)))
)

# not masking out protected areas
# # create a mask out of wdpa layer
# # to remove already protected areas from manageable carbon
# wdpa_mask = ee.Image.constant(1).clip(y2y_wdpa).mask()

In [15]:
# compute zonal statistics across ecosystems
# create dictionary with masks
mask_dict = {
    'Temperate Forest':temperate_forest_mask, 
    'Northern Forest':northern_forest_mask, 
    'Temperate Grass Shrub':temperate_grass_shrub_mask,
    'Montane Grass Shrub':montane_grass_shrub_mask, 
    'Northern Grass Shrub':northern_grass_shrub_mask, 
    'Temperate Wetlands':temperate_wetlands_mask,
    'Northern Wetlands':northern_wetlands_mask, 
    'Temperate Peatlands':temperate_peatlands_mask, 
    'Northern Peatlands':northern_peatlands_mask,
    'Temperate Barren':temperate_barren_mask, 
    'Northern Barren':northern_barren_mask
    }

zonal_stats_eco = ee.FeatureCollection([])

# loop through ecosystem masks
for name, mask in mask_dict.items():

    # set input parameters
    y2y = ee.FeatureCollection("projects/ee-bermane/assets/y2y")
    roi = y2y
    filter_roi = False
    clip_to_fc = False
    mask_with_img = True
    img = mask
    property_name = ""
    property_values = [""]
    annual_emissions = False
    annual_averages = True

    # run function
    _, zonal_stats_fc = calc_carbon_sequestration(
        roi=y2y,
        clip_to_fc=clip_to_fc,
        mask_with_img=mask_with_img,
        img=img,
        filter_roi=filter_roi,
        property_name=property_name,
        property_values=property_values,
        annual_emissions=annual_emissions,
        annual_averages=annual_averages
    )

    # set ecosystem name as property of feature
    zonal_stats_fc = zonal_stats_fc.map(lambda f: f.set('ecosystem', name))

    # add to global fc
    zonal_stats_eco = zonal_stats_eco.merge(zonal_stats_fc)
    
# export csv to drive
filename = "y2y_ecosystem_carbon_sequestration"

task = ee.batch.Export.table.toDrive(
    collection=zonal_stats_eco,
    description=filename,
    fileNamePrefix=filename,
    fileFormat='CSV'
)
task.start()

## Export rasters

In [9]:
# load layers and mosaic
# set to mg/ha of carbon
# set to annual values

y2y = ee.FeatureCollection("projects/ee-bermane/assets/y2y")

emissions = ee.Image(
        "projects/forma-250/assets/gfw_forest_carbon_gross_emissions/gross_emissions_forest_extent_per_hectare_v1_4_2_2001_2024"
    ).multiply(12/44).divide(24)

removals = ee.Image(
        "projects/forma-250/assets/gfw_forest_carbon_gross_removals/gross_removals_forest_extent_per_hectare_v1_4_2_2001_2024"
    ).multiply(12/44).divide(24)

netflux = ee.Image(
        "projects/forma-250/assets/gfw_forest_carbon_net_flux/net_flux_forest_extent_per_hectare_v1_4_2_2001_2024"
    ).multiply(12/44).divide(24)

# load export crs info
out_proj = ee.Image(
        "projects/forma-250/assets/gfw_forest_carbon_gross_emissions/gross_emissions_forest_extent_per_hectare_v1_4_2_2001_2024"
    ).projection().getInfo()

out_scale = ee.Image(
        "projects/forma-250/assets/gfw_forest_carbon_gross_emissions/gross_emissions_forest_extent_per_hectare_v1_4_2_2001_2024"
    ).projection().nominalScale().getInfo()

# check scale
print('Mosaicked Scale: ', emissions.projection().nominalScale().getInfo())
print('Original Scale: ', out_scale)

Mosaicked Scale:  27.829872698318393
Original Scale:  27.829872698318393


In [ ]:
# export rasters to disk in native resolution
data_folder = './carbon_flux_data/output_layers/'

# geemap.download_ee_image(emissions, 
#                          filename=data_folder + 'emissions_t_ha_yr.tif', 
#                          region=y2y.geometry(),
#                          crs=out_proj.get('crs'),
#                          scale=out_scale)

# geemap.download_ee_image(removals, 
#                          filename=data_folder + 'removals_t_ha_yr.tif', 
#                          region=y2y.geometry(),
#                          crs=out_proj.get('crs'),
#                          scale=out_scale)

# geemap.download_ee_image(netflux, 
#                          filename=data_folder + 'netflux_t_ha_yr.tif', 
#                          region=y2y.geometry(),
#                          crs=out_proj.get('crs'),
#                          scale=out_scale)


Consider adjusting `region`, `scale` and/or `dtype` to reduce the emissions_t_ha_yr.tif download size (raw: 107.14 GB).


emissions_t_ha_yr.tif: |          | 0.00/107G (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: None
Consider adjusting `region`, `scale` and/or `dtype` to reduce the removals_t_ha_yr.tif download size (raw: 107.14 GB).


removals_t_ha_yr.tif: |          | 0.00/107G (raw) [  0.0%] in 00:00 (eta:     ?)

Consider adjusting `region`, `scale` and/or `dtype` to reduce the netflux_t_ha_yr.tif download size (raw: 107.14 GB).


netflux_t_ha_yr.tif: |          | 0.00/107G (raw) [  0.0%] in 00:00 (eta:     ?)